**Wikipedia spanish language model** 

In [ ]:
!pip install -q gwpy

In [ ]:
!git clone https://github.com/fastai/fastai2.git 

In [ ]:
!pip install fastai2

In [3]:
from google.colab import drive
import zipfile
import os
import re
from pathlib import Path
from fastai2.text.all import *
import dill as pickle

# Data preparation (Only run to create pickle file)

Connecting drive, train and test set organization, tokenization, numericalization and creation of LMDataLoader

In [ ]:
drive.mount('/content/drive/')

In [5]:
#to not show cell output-- too long
%%capture
!unzip -q "/content/drive/My Drive/Colab Notebooks/TLAC/wiki_art.zip"

In [6]:
def set_organization(dset):
  
  path="/content/wiki_art/{}".format(dset)

  for filename in os.listdir(path):
    os.rename(os.path.join(path, filename), os.path.join(path, re.sub('[^A-Za-z0-9]+(?![^.]*$)', '',filename)))

  n_files = len(list(os.listdir(path)))
  !ls $path
  print("elements_{}_set={} ".format(dset,n_files))


In [ ]:
%%capture
set_organization("train")

In [43]:
%%capture
set_organization("test")

In [ ]:
path = Path('/content/wiki_art')
path.ls()

In [ ]:
texts = get_files(path, extensions=['.txt'], folders=['train', 'test'])

In [11]:
def read_file(f): return L(f.read().split(' '))

In [ ]:
splits = RandomSplitter(valid_pct=0.1)(texts)
tfms = [Tokenizer.from_folder(path), Numericalize()]
dsets = Datasets(texts, [tfms], splits=splits, dl_type=LMDataLoader)

In [44]:
pickle.dump(dsets, open(f'/content/dsets.pkl','wb'))

# Language model creation



In [38]:
dsets = pickle.load(open(f'/content/dsets.pkl','rb'))

In [45]:
bs,sl=64,80
dbunch_lm = dsets.dataloaders(bs=bs, seq_len=sl, val_bs=bs)

In [ ]:
dbunch_lm.show_batch()

In [47]:
learn = language_model_learner(dbunch_lm, AWD_LSTM, drop_mult=0.3)

In [ ]:
learn.fit_one_cycle(1, 1e-2)